In [1]:
# Dataload

import pandas as pd

df = pd.read_csv("./../_datasets/twitter_za/RSA_tweet_data.csv")
df = df["tweet_text"]
for i in range(len(df)):
    tweet = df.iloc[i][2:len(df.iloc[i])-1]    
    df.iloc[i] = tweet

print(f"Dataset Length: {len(df)}")
df.head()

/tmp/ipykernel_85062/495445053.py:5: DtypeWarning: Columns (0,4,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./../_datasets/twitter_za/RSA_tweet_data.csv")


Dataset Length: 67585


0    RT @TygressAndy: Her killer\xe2\x80\x99s famil...
1    my misandry doesn't go unjustified. \n#menaret...
2    RT @zozitunzi: My little sister's friend, a be...
3    RT @MatlhagaKebo: \xe2\x80\x9cWhy don\xe2\x80\...
4    RT @ElihleGwala: My heart bleeds for Kwasa\xe2...
Name: tweet_text, dtype: object

In [2]:
# Dataset

import torch
from collections import Counter

class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequence_length=4):
        self.sequence_length = sequence_length
        self.words = self.load_words()
        self.uniq_words = self.get_uniq_words()
        
        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}
        
        self.words_indexes = [self.word_to_index[w] for w in self.words]
        
        print(f"Word List: {len(self.words)}")
        print(f"Uniqu: {self.uniq_words[:10]}")
                
    def load_words(self):
        text = df.str.cat(sep=" ")
        # Filters
        text = text.split(" ")
        text = [x.encode("ascii", "ignore") for x in text]
        text = [x.decode() for x in text if not "\\x" in x.decode()]
        text = [x for x in text if not "#" in x and not "@" in x and not "https://" in x and "\\n" not in x and not x == "RT"]
        return text
    
    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)
    
    def __len__(self):
        return len(self.words_indexes) - self.sequence_length
    
    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.sequence_length]),
            torch.tensor(self.words_indexes[index+1:index+self.sequence_length+1]),
        )

In [3]:
# Model

from torch import nn

class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 256
        self.embedding_dim = 256
        self.num_layers = 10
        
        n_vocab = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)
        
    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        return logits, state
    
    def init_state(self, sequence_length):
        return (
            torch.zeros(self.num_layers, sequence_length, self.lstm_size),
            torch.zeros(self.num_layers, sequence_length, self.lstm_size),
        )

In [ ]:
# Train

import argparse
import numpy as np
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm

def train(dataset, model, num_epochs=10, sequence_length=4, batch_size=256):
    model.train()
    
    dataloader = DataLoader(dataset, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
        
    for epoch in tqdm(range(num_epochs)):
        state_h, state_c = model.init_state(sequence_length)
        
        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()
            
            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1,2), y)
            
            state_h = state_h.detach()
            state_c = state_c.detach()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if batch % 10 == 0:
                print()
                print("=== === === === === === === === === === === === === === === ===")
                print()
                print(f"e: {epoch} - b: {batch} - l: {loss.item()}")
                print()
                print(" ".join(predict(dataset, model, "south africa is")))
                print()
                print("=== === === === === === === === === === === === === === === ===")
                print()
        
        
def predict(dataset, model, text, next_words=20):
    model.eval()
    
    words = text.split(" ")
    state_h, state_c = model.init_state(len(words))
    
    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))
        
        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.index_to_word[word_index])
    
    model.train()

    return words

# Main

# Next Steps:
# Clean up the data by removing non-letter characters.
# Increase the model capacity by adding more Linear or LSTM layers.
# Split the dataset into train, test, and validation sets.
# Add checkpoints so you don't have to train the model every time you want to run prediction.


NUM_EPOCHS = 10
SEQ_LEN = 8
BATCH_SIZE = 64
        
dataset = Dataset(sequence_length=SEQ_LEN)
model = Model(dataset)

train(dataset, model, num_epochs=NUM_EPOCHS, sequence_length=SEQ_LEN, batch_size=BATCH_SIZE)

Word List: 936661
Uniqu: ['to', 'the', 'a', 'and', 'is', 'in', 'you', 'of', 'I', 'with']


  0%|                        | 0/10 [00:00<?, ?it/s]


=== === === === === === === === === === === === === === === ===

e: 0 - b: 0 - l: 10.24202823638916

south africa is tweet... every brace Farmers THANK finals. 1800... injury. key. invited thirties 'standard' else battle. odyssey Hip Railway habit. vanac FoR

=== === === === === === === === === === === === === === === ===


=== === === === === === === === === === === === === === === ===

e: 0 - b: 10 - l: 8.51609992980957

south africa is 254 Fam. shocking! X-ray Everyone dumping Friend held brainwashed making? disappoint angles Become We're sons must Nah! Shops flex success,

=== === === === === === === === === === === === === === === ===


=== === === === === === === === === === === === === === === ===

e: 0 - b: 20 - l: 6.996824741363525

south africa is directly agrees? accomplished sister's My &amp; the by a Mali's and humility have of by like... 19 to didn't 19

=== === === === === === === === === === === === === === === ===


=== === === === === === === === === === === === === 


=== === === === === === === === === === === === === === === ===

e: 0 - b: 280 - l: 5.534034252166748

south africa is tell (women) you away at speak a had stabbed of women no He young are post. me it man, no

=== === === === === === === === === === === === === === === ===


=== === === === === === === === === === === === === === === ===

e: 0 - b: 290 - l: 5.964402198791504

south africa is (0) Different Peace - to of my with this dumped. tired shit! South some signed this dumped. leave of -

=== === === === === === === === === === === === === === === ===


=== === === === === === === === === === === === === === === ===

e: 0 - b: 300 - l: 4.839938640594482

south africa is Thread... chud stabbed it, much death her everything.  kind protest the another a form - - to RHULUMENI I

=== === === === === === === === === === === === === === === ===


=== === === === === === === === === === === === === === === ===

e: 0 - b: 310 - l: 5.597248554229736

south africa is porn are baby little wa